# Load Library

In [3]:
import os
import random
import re
import traceback
import warnings
from glob import glob
from pprint import pprint
from textwrap import dedent
from typing import List, Tuple

import gradio as gr
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

warnings.filterwarnings("ignore")

load_dotenv()


warnings.filterwarnings('ignore')

load_dotenv()

%reload_ext autotime

ModuleNotFoundError: No module named 'autotime'

In [2]:
DATA_DIR = '../data'
PERSIST_DIR = "./chroma_db"

# 법률 이름 매핑 딕셔너리
LAW_NAME_MAPPING = {
    "고용보험법": "employment_insurance_act",
    "근로기준법": "labor_standards_act",
    "남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률": "gender_equality_employment_act",
}

# Define Function

In [4]:
def split_articles(lines):
    """
    조문 본문에서 개행으로 인한 단어 분리를 방지하기 위해
    줄 간 공백 없이 연결한다. 조문은 '제X조(제목)' 형식으로 시작한다.
    """
    # 조문을 저장할 리스트
    articles = []
    # 현재 처리 중인 조문의 제목
    current_title = None  
    # 현재 처리 중인 조문의 내용
    current_content = []

    # '제X조(제목)' 형식을 찾기 위한 정규표현식 패턴
    title_pattern = re.compile(r'^(제\d+조(?:의\d+)?\([^)]+\))')

    # 각 줄을 순회하며 처리
    for line in map(str.strip, lines):
        if not line:  # 빈 줄 건너뛰기
            continue

        # 조문 제목 패턴이 매칭되는지 확인
        match = title_pattern.match(line)
        if match:
            # 이전 조문이 있으면 저장
            if current_title:
                articles.append(f"{current_title}\n{''.join(current_content)}")
            # 새로운 조문 시작
            current_title = match.group(1)
            rest = line[len(current_title):].strip()
            current_content = [rest] if rest else []
        elif current_title:
            # 조문 내용 추가
            current_content.append(line)

    # 마지막 조문 저장
    if current_title:
        articles.append(f"{current_title}\n{''.join(current_content)}")

    return [a.strip() for a in articles]

def parse_law_with_sections(law_text):
    """
    전체 법령 텍스트를 서문, 장, 절, 조문 단위로 구조화하여 파싱한다.
    """
    # 줄바꿈 문자 정규화
    law_text = law_text.replace('\r\n', '\n').replace('\r', '\n')
    lines = map(str.strip, law_text.split('\n'))

    # 파싱 결과를 저장할 딕셔너리
    parsed_law = {'서문': '', '장': {}, '부칙': ''}
    
    # 장과 절을 찾기 위한 정규표현식 패턴
    chapter_pattern = re.compile(r'^제\d+장\s+\S+')
    section_pattern = re.compile(r'^제\d+절\s+\S+')

    # 현재 처리 중인 위치 추적용 변수들
    current_chapter = None
    current_section = None
    chapter_buffer = {}
    buffer = []
    preamble_done = False

    # 각 줄을 순회하며 처리
    for i, line in enumerate(lines):
        if not line:
            continue

        # 부칙 처리
        if line.startswith("부칙"):
            if current_section:
                chapter_buffer[current_section] = buffer
            elif buffer:
                chapter_buffer['조문'] = buffer
            if current_chapter:
                parsed_law['장'][current_chapter] = {
                    k: split_articles(v) for k, v in chapter_buffer.items()
                }
            parsed_law['부칙'] = '\n'.join([line] + list(lines)[i+1:]).strip()
            break

        # 장 처리
        if chapter_pattern.match(line):
            if not preamble_done:
                parsed_law['서문'] = '\n'.join(buffer).strip()
                preamble_done = True
            elif current_chapter:
                if current_section:
                    chapter_buffer[current_section] = buffer
                elif buffer:
                    chapter_buffer['조문'] = buffer
                parsed_law['장'][current_chapter] = {
                    k: split_articles(v) for k, v in chapter_buffer.items()
                }
            current_chapter = line
            chapter_buffer = {}
            current_section = None
            buffer = []
            continue

        # 절 처리
        if section_pattern.match(line):
            if current_section:
                chapter_buffer[current_section] = buffer
            elif buffer:
                chapter_buffer['조문'] = buffer
            current_section = line
            buffer = []
            continue

        buffer.append(line)

    # 마지막 장 저장
    if current_chapter:
        if current_section:
            chapter_buffer[current_section] = buffer
        elif buffer:
            chapter_buffer['조문'] = buffer
        parsed_law['장'][current_chapter] = {
            k: split_articles(v) for k, v in chapter_buffer.items()
        }

    return parsed_law


def create_document(pdf_file: str, law_name: str) -> List[Document]:
    loader = PyPDFLoader(pdf_file)
    pages = loader.load()
    
    # 푸터 제거용 패턴
    footer_pattern = rf"법제처\s+\d+\s+국가법령정보센터\s*\n{law_name}"
    re_footer = re.compile(footer_pattern)
    
    # 각 페이지에서 푸터 제거
    cleaned_pages = []
    for p in pages:
        text = p.page_content.strip()
        cleaned = re_footer.sub('', text)
        cleaned_pages.append(cleaned.strip())
        
    # 전체 문서 텍스트 결합
    law_text = "\n".join(cleaned_pages)
    
    parsed_law = parse_law_with_sections(law_text)
    
    # Document 객체로 만들기
    final_docs = []
    
    for chapter, sections in parsed_law["장"].items():
        for section, articles in sections.items():
            for article_text in articles:
                # 조문 제목 줄 (예: "제1조(목적)")
                title_line = article_text.split("\n", 1)[0]
                
                # LangChain 문서 metadata 구성
                metadata = {
                    "source": pdf_file,
                    "chapter": chapter,
                    "section": section,
                    "name": law_name,
                    "article": title_line
                }
                
                # LangChain 문서 본문 구성
                content = (
                    f"[법률정보]\n"
                    f"다음 조항은 {metadata['name']} {metadata['chapter']} {metadata['section']}에서 발췌한 내용입니다.\n\n[법률조항]\n"
                    f"{article_text}"
                )

                final_docs.append(Document(page_content=content, metadata=metadata))
                
    return final_docs

# Load Raw Documents

In [5]:
# pdf 파일 목록을 확인
pdf_files = glob(os.path.join(DATA_DIR, '*.pdf'))

pdf_files

['../data\\고용보험법(법률)(제20519호)(20250223).pdf',
 '../data\\근로기준법(법률)(제20520호)(20250223).pdf',
 '../data\\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).pdf']

# Create VectorDB

In [7]:
# embeddings = OpenAIEmbeddings()

for idx, (law_name, collection_name) in enumerate(LAW_NAME_MAPPING.items()):
    final_documents = create_document(pdf_file=pdf_files[idx], law_name=law_name)
    
    # Chroma.from_documents(
    #     documents=final_documents,
    #     embedding=embeddings,
    #     collection_name=collection_name,
    #     persist_directory=PERSIST_DIR
    # )
    

In [14]:
final_documents

[Document(metadata={'source': '../data\\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).pdf', 'chapter': '제1장 총칙 <개정 2007. 12. 21.>', 'section': '조문', 'name': '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률', 'article': '제1조(목적)'}, page_content='[법률정보]\n다음 조항은 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제1장 총칙 <개정 2007. 12. 21.> 조문에서 발췌한 내용입니다.\n\n[법률조항]\n제1조(목적)\n이 법은 「대한민국헌법」의 평등이념에 따라 고용에서 남녀의 평등한 기회와 대우를 보장하고 모성 보호와여성 고용을 촉진하여 남녀고용평등을 실현함과 아울러 근로자의 일과 가정의 양립을 지원함으로써 모든 국민의 삶의 질 향상에 이바지하는 것을 목적으로 한다.[전문개정 2007. 12. 21.]'),
 Document(metadata={'source': '../data\\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).pdf', 'chapter': '제1장 총칙 <개정 2007. 12. 21.>', 'section': '조문', 'name': '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률', 'article': '제2조(정의)'}, page_content='[법률정보]\n다음 조항은 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제1장 총칙 <개정 2007. 12. 21.> 조문에서 발췌한 내용입니다.\n\n[법률조항]\n제2조(정의)\n이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017. 11. 28., 2020. 5. 26.>1. “차별”이란 사업주가 근로자에게 성별, 혼인, 가족 안에서의 지위, 임신 또는 출산 등의 사유로 합리적인 이유 없이 채용 또는 근로의 조건을 다르게 하거나 그 밖의 불리한 조치를 하는 경우[사업주가 채용조건이

# Define Tool

In [6]:
ei_db = Chroma(
    embedding_function=embeddings,
    collection_name=LAW_NAME_MAPPING.get("고용보험법"),
    persist_directory=PERSIST_DIR
)

ls_db = Chroma(
    embedding_function=embeddings,
    collection_name=LAW_NAME_MAPPING.get("근로기준법"),
    persist_directory=PERSIST_DIR
)

ge_db = Chroma(
    embedding_function=embeddings,
    collection_name=LAW_NAME_MAPPING.get("남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률"),
    persist_directory=PERSIST_DIR
)

time: 453 ms (started: 2025-04-29 14:59:15 +09:00)


In [7]:
@tool
def search_ei_law(query: str) -> List[Document]:
    """
    Searches the Employment Insurance Act (고용보험법) database for relevant legal articles.

    Args:
        query (str): A keyword or phrase related to the Employment Insurance Act to search for.

    Returns:
        List[Document]: 
            - A list of Document objects containing the most relevant sections.
            - If no matching articles are found, returns a Document indicating "검색 결과가 없습니다."

    Usage:
        Use this tool when you need to retrieve legal information, specific clauses, 
        or general background knowledge about the Employment Insurance Act.
    """
    docs = ei_db.similarity_search(query, k=2)
    return docs if docs else [Document(page_content="", metadata={"error": "검색 결과가 없습니다."})]

@tool
def search_ls_law(query: str) -> List[Document]:
    """
    Searches the Labor Standards Act (근로기준법) database for relevant legal articles.

    Args:
        query (str): A keyword or phrase related to the Labor Standards Act to search for.

    Returns:
        List[Document]: 
            - A list of Document objects containing the most relevant sections.
            - If no matching articles are found, returns a Document indicating "검색 결과가 없습니다."

    Usage:
        Use this tool when you need to retrieve legal information, specific clauses, 
        or general background knowledge about the Labor Standards Act.
    """
    docs = ls_db.similarity_search(query, k=2)
    return docs if docs else [Document(page_content="", metadata={"error": "검색 결과가 없습니다."})]

@tool
def search_ge_law(query: str) -> List[Document]:
    """
    Searches the Gender Equality Employment Act 
    (남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률) database for relevant legal articles.

    Args:
        query (str): A keyword or phrase related to the Gender Equality Employment Act to search for.

    Returns:
        List[Document]: 
            - A list of Document objects containing the most relevant sections.
            - If no matching articles are found, returns a Document indicating "검색 결과가 없습니다."

    Usage:
        Use this tool when you need to retrieve legal information, specific clauses, 
        or general background knowledge about the Gender Equality Employment Act.
    """
    docs = ge_db.similarity_search(query, k=2)
    return docs if docs else [Document(page_content="", metadata={"error": "검색 결과가 없습니다."})]

time: 16 ms (started: 2025-04-29 14:59:16 +09:00)


# LangChain Agent

In [8]:
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""You are an AI legal assistant specializing in Korean labor-related laws.
                      Your primary mission is to provide users with accurate, reliable, and contextually appropriate legal information based on three major legislations.
                      
                      Key guidelines:
                      1. For questions related to general labor rights, working conditions, employment contracts, wages, working hours, minimum wage, and termination procedures, use the search_ls_law tool.
                        - Related keywords: "근로시간", "연장근로", "임금", "최저임금", "해고", "취업규칙", "연차휴가", "근로계약서", "퇴직금"
                        - This tool accesses the Labor Standards Act (근로기준법).
                      2. For questions regarding unemployment benefits, employment insurance enrollment, premium payments, claims, eligibility, and job protection during unemployment, use the search_ei_law tool.
                        - Related keywords: "실업급여", "고용보험", "피보험자", "보험료", "이직확인서", "재취업활동"
                        - This tool accesses the Employment Insurance Act (고용보험법).
                      3. For questions involving gender equality at work, maternity and paternity leave, prevention of workplace discrimination, childcare leave, and work-family balance policies, use the search_ge_law tool.
                        - Related keywords: "출산휴가", "육아휴직", "남녀차별", "임신기 근로시간 단축", "배우자 출산휴가", "가족돌봄휴가", "직장 내 성희롱 예방"
                        - This tool accesses the Gender Equality Employment Act (남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률).
                      4. If a user's query is ambiguous or covers multiple topics (e.g., "육아휴직 중 실업급여 수급 가능 여부"), intelligently select multiple tools, retrieve information from each relevant law, and combine the results into a coherent answer.
                      5. If necessary information is missing, politely ask the user to clarify their question to ensure correct tool usage.
                      6. Always prioritize delivering clear, concise, and legally accurate information based on the search results.
                      7. Simplify legal jargon into plain, user-friendly language unless the user specifically requests detailed legal references.
                      8. Maintain a professional, supportive, and trustworthy tone. Users may be seeking crucial legal guidance, so be respectful, empathetic, and clear.
                      9. If no relevant documents are found in the database, explicitly inform the user and suggest further actions if appropriate.
                      10. Ensure final responses are natural and conversational, maintaining a helpful chatbot-like communication style while retaining professional accuracy.
                      
                      Remember:
                        - Understand the scope of each tool precisely.
                        - Match the user's query keywords to the right tool(s).
                        - Combine tool results effectively for complex questions.
                        - Always aim to deliver the most complete, accurate, and user-centered answers possible.
                    """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

time: 0 ns (started: 2025-04-29 14:59:16 +09:00)


In [9]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [search_ls_law, search_ei_law, search_ge_law]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

time: 1.98 s (started: 2025-04-29 14:59:16 +09:00)


In [10]:
query = "육아휴직을 신청할 수 있는 조건이 있나요?"
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_ge_law` with `{'query': '육아휴직'}`


[Document(metadata={'article': '제19조의6(직장복귀를 위한 사업주의 지원)', 'chapter': '제3장 모성 보호 <개정 2007. 12. 21.>', 'name': '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률', 'section': '조문', 'source': '../data\\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).pdf'}, page_content='[법률정보]\n다음 조항은 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제3장 모성 보호 <개정 2007. 12. 21.> 조문에서 발췌한 내용입니다.\n\n[법률조항]\n제19조의6(직장복귀를 위한 사업주의 지원)\n사업주는 이 법에 따라 육아휴직 중인 근로자에 대한 직업능력 개발 및 향상을 위하여 노력하여야 하고 출산전후휴가, 육아휴직 또는 육아기 근로시간 단축을 마치고 복귀하는 근로자가 쉽게직장생활에 적응할 수 있도록 지원하여야 한다. <개정 2012. 2. 1.>[본조신설 2007. 12. 21.]'), Document(metadata={'article': '제19조의6(직장복귀를 위한 사업주의 지원)', 'chapter': '제3장 모성 보호 <개정 2007. 12. 21.>', 'name': '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률', 'section': '조문', 'source': '../data\\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).pdf'}, page_content='[법률정보]\n다음 조항은 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제3장 모성 보호 <개정 2007. 12. 21.> 조문에서 발췌한 내용입니다.\n\n[법률조항]\n제19조의6(직장복귀를 위한 사업주의 지원)\n사업

In [11]:
pprint(agent_response)

{'input': '육아휴직을 신청할 수 있는 조건이 있나요?',
 'output': '육아휴직을 신청할 수 있는 조건은 다음과 같습니다:\n'
           '\n'
           '1. **근로자 자격**: 육아휴직은 근로자가 자녀를 양육하기 위해 신청할 수 있는 권리입니다. 일반적으로, 자녀가 만 '
           '8세 이하이거나 초등학교 2학년 이하인 경우에 해당합니다.\n'
           '\n'
           '2. **근속 기간**: 육아휴직을 신청하기 위해서는 일정 기간 이상 근무해야 할 수 있습니다. 보통 6개월 이상의 '
           '근속이 요구됩니다.\n'
           '\n'
           '3. **신청 절차**: 육아휴직을 신청하려면, 고용주에게 사전에 신청서를 제출해야 하며, 필요한 서류를 함께 제출해야 '
           '합니다.\n'
           '\n'
           '4. **사업주의 지원**: 사업주는 육아휴직 중인 근로자가 직장에 복귀할 수 있도록 지원해야 하며, 직업능력 개발 및 '
           '향상을 위한 노력을 해야 합니다.\n'
           '\n'
           '이 외에도 구체적인 조건이나 절차는 고용주나 회사의 취업규칙에 따라 다를 수 있으므로, 해당 사항을 확인하는 것이 '
           '중요합니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!'}
time: 0 ns (started: 2025-04-29 14:59:24 +09:00)


# Gradio Demo

In [ ]:
ERROR_RESPONSES = [
    "⚠️ 죄송합니다. 다시 시도해 주세요.",
    "⚠️ 문제가 발생했습니다. 곧 복구하겠습니다.",
    "⚠️ 시스템이 응답하지 않았어요. 잠시 후 재시도 바랍니다."
]

# 대화 이력을 LangChain 형식으로 변환
def build_chat_history(history: List[Tuple[str, str]]) -> List:
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))
    return chat_history

# 에이전트 호출 및 응답 처리
def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        chat_history = build_chat_history(history)[-2:]
        print(f"[DEBUG] chat_history: {chat_history}")

        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history,
        })
        print(f"[DEBUG] response: {response}")

        return response.get("output", "응답이 없습니다.")
    except Exception as e:
        print(f"[Error] {e}")
        traceback.print_exc()  # <-- 에러 위치까지 같이 출력
        return random.choice(ERROR_RESPONSES)


# 예제 질문 리스트
example_questions = [
    "육아휴직을 신청할 수 있는 조건이 있나요?",
    "퇴직금은 언제 받을 수 있나요?",
    "출산휴가 중 실업급여를 받을 수 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="육아휴직 관련 AI 어시스턴트",
    description="근로기준법, 고용보험법, 남녀고용평등법 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft(),
)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0953399e9bfc2a26da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


time: 5.25 s (started: 2025-04-29 14:59:24 +09:00)


In [13]:
demo.close()

Closing server running on port: 7860
time: 140 ms (started: 2025-04-29 14:59:29 +09:00)
